!earthengine authenticate

In [1]:
import geopandas as gpd
import ee
import geemap
import numpy as np
import rasterio
import time
# from sklearn.metrics import confusion_matrix
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from treeinterpreter import treeinterpreter as ti

In [2]:
ee.Initialize()

In [3]:
#shp path
tp_crop ='crop_pt.shp'
tp_non_crop = 'non_crop_pt.shp'

#ee object
crop_ee = geemap.shp_to_ee(tp_crop)
noncrop_ee = geemap.shp_to_ee(tp_non_crop)
#feature collection
fc = crop_ee.merge(noncrop_ee)
#define bounding box
bbox = ee.Geometry.bounds(fc)
aoi = bbox
#TODO #read about system.index

In [10]:
sdate= '2020-04-01'
edate= '2020-08-31'
clmasl= 5
#cloud mask function
def clmask(image):
    qa = image.select('QA60')
    # cloudBitMask = 1 << 10
    # cirrusBitMask = 1 << 11

    # mask = qa.bitwiseAnd(cloudBitMask).eq(0)
    # mask = qa.bitwiseAnd(cirrusBitMask).eq(0)
    return image.updateMask(qa.lt(1)).divide(10000)


imgcollection = ee.ImageCollection("COPERNICUS/S2")\
    .filterBounds(aoi)\
        .filterDate(sdate, edate)\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', clmasl))\
                .filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT',clmasl))\
                    .map(clmask)
                    
image = imgcollection.median()

In [11]:
#get band properties
band3 = image.select('B3')
proj = band3.projection().getInfo()
crs = proj['crs']
#resample the resolution
band20m = image.select('B11', 'B12')
swir_b10m = band20m.resample('bilinear').reproject(crs=crs, scale=10.0).rename('B11r', 'B12r')
image = image.addBands(swir_b10m)


#Vegetation Index 
#NDVI
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
image = image.addBands(ndvi)


#select bands needed for the classification including NDVI
band_sl = ('B2', 'B3', 'B4', 'B8', 'B11r', 'B12r', 'NDVI')

#dataset for classification
image_feature = image.select(band_sl)

image_feature = image_feature.clip(aoi)
# print(image_feature.select('B11r').projection().nominalScale().getInfo())

In [15]:
#exprot to assest - Need review\not working
task = ee.batch.Export.image.toAsset(**{'image':image_feature,
'description': 'Compositeband',
'assetId':'users/aadebowaledaniel/binaryimage_ft', 
'scale': 10, 
'region':aoi.getInfo()['coordinates'],
'maxPixels': 1e13,
})
task.start()


In [16]:
#Monitor the task
while task.status()['state'] in ['READY', 'RUNNING']:
  print(task.status())
  time.sleep(5)
else:
  print(task.status())

4JTL7QPLQPKARM'}
{'state': 'RUNNING', 'description': 'Compositeband', 'creation_timestamp_ms': 1620132427693, 'update_timestamp_ms': 1620133943572, 'start_timestamp_ms': 1620132438941, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': '3QIRJXI5GF4JTL7QPLQPKARM', 'name': 'projects/earthengine-legacy/operations/3QIRJXI5GF4JTL7QPLQPKARM'}
{'state': 'RUNNING', 'description': 'Compositeband', 'creation_timestamp_ms': 1620132427693, 'update_timestamp_ms': 1620133943572, 'start_timestamp_ms': 1620132438941, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': '3QIRJXI5GF4JTL7QPLQPKARM', 'name': 'projects/earthengine-legacy/operations/3QIRJXI5GF4JTL7QPLQPKARM'}
{'state': 'RUNNING', 'description': 'Compositeband', 'creation_timestamp_ms': 1620132427693, 'update_timestamp_ms': 1620133943572, 'start_timestamp_ms': 1620132438941, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': '3QIRJXI5GF4JTL7QPLQPKARM', 'name': 'projects/earthengine-legacy/operations/3QIRJXI5GF4JTL7QPLQPKARM'}
{'state': 'RUNNING', 'd

In [10]:
#import composite from asset
predimage = ee.Image('users/aadebowaledaniel/binaryimage_ft')

In [12]:
predbands = image_feature.bandNames()

In [13]:
#continue from here
trainingSample  = image_feature.sampleRegions(
    collection = fc,
    properties=['label'],
    scale = 10, 
    geometries = True
)
# trainingSample.size().getInfo()

rantraning = trainingSample.randomColumn('random')
split = 0.7
trainsample = rantraning.filter(ee.Filter.lt('random', split))
testsample = rantraning.filter(ee.Filter.gte('random', split))

#train random forest classifier
classifier = ee.Classifier.smileRandomForest(100).train(trainsample, 'label', predbands)


#confusion matrix train Accurracy
trainAccuracy = classifier.confusionMatrix()
print('Resubstitution error matrix: ', trainAccuracy.getInfo())
print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())

#classify features
testclassifier = testsample.classify(classifier)
#cconfusion matrix representing expected accuracy
testAccuracy = testclassifier.errorMatrix('label', 'classification')
print('confusion Matrix', testAccuracy.getInfo())
print('Valdation overall accuracy:', testAccuracy.accuracy().getInfo())

#classify image
Imageclassified = image_feature.classify(classifier)
# altImageclassified = predimage.classify(classifier)

Resubstitution error matrix:  [[436, 10], [0, 1792]]
Training overall accuracy:  0.9934668989547039
confusion Matrix [[142, 59], [29, 730]]
Valdation overall accuracy: 0.8964467005076142


In [14]:
Map = geemap.Map(center=(0.9613,34.6536),zoom=8)
Map

Map(center=[0.9613, 34.6536], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [15]:
Map.addLayer(image_feature, {'bands': ['B4', 'B3', 'B2'], 'min': 0.0, 'max': 0.3, 'gamma': 2}, 'True colour composite')

In [16]:
vis_params = {'min': 0, 'max':1, 'pallete':['cdb33b', #1
            'CEBD15', #0
            ]}
#{'min':0,'max':1,'palette':["green","white"]}            
Map.addLayer(Imageclassified,vis_params, 'Classified')

In [23]:
#convert to Vector
cropmask = Imageclassified.eq(1)
cropmask = cropmask.updateMask(cropmask)

cropmaskVec = cropmask.reduceToVectors({
    'geometryType': 'polygon',
  'reducer': ee.Reducer.countEvery(),
  'scale': 10,
  'geometry':aoi,
})

In [ ]:
# def clmask(image):
#     qa = image.select('QA60')
#     cloudBitMask = 1 << 10
#     cirrusBitMask = 1 << 11

#     mask = qa.bitwiseAnd(cloudBitMask).eq(0)
#     mask = qa.bitwiseAnd(cirrusBitMask).eq(0)
#     # return image.updateMask(qa.lt(1)).divide(10000)
#     return image.updateMask(mask.lt(1)).divide(10000)